In [106]:
import json
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.models import Sequential
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
#pip3 install bpmll
#using bpmll in the loss
from bpmll import bp_mll_loss
from sklearn.preprocessing import MinMaxScaler,StandardScaler
import pandas as pd

In [2]:
train_data = json.load(open('train.json'))

In [3]:
train_data[0].keys()

dict_keys(['authors', 'year', 'abstract', 'venue', 'title'])

In [4]:
train_data[0]

{'authors': [42, 13720, 36],
 'year': 9,
 'abstract': [2455,
  1858,
  2335,
  1543,
  1800,
  1860,
  2000,
  2867,
  1546,
  1874,
  2059,
  1525,
  2590,
  4196,
  12,
  2634,
  1543,
  1800,
  1586,
  2866,
  3595,
  1866,
  1670,
  2000,
  3743,
  1542,
  1650,
  1527,
  33,
  4407,
  1543,
  1535,
  1962,
  1961,
  1543,
  33,
  1700,
  1543,
  1535,
  1647,
  1546,
  1580,
  4720,
  12,
  1731,
  4231,
  2601,
  1553,
  1704,
  1605,
  2456,
  1543,
  3281,
  1594,
  4407,
  2168,
  1542,
  1586,
  3781,
  2471,
  1525,
  1859,
  1669,
  2512,
  4572,
  1546,
  1609,
  3781,
  2471,
  1525,
  3393,
  12,
  37,
  1712,
  1586,
  4196,
  1650,
  1527,
  3281,
  1594,
  4407,
  1800,
  4708,
  1904,
  2059,
  2411,
  12],
 'venue': 20,
 'title': [41,
  1550,
  1563,
  1594,
  1544,
  1919,
  1644,
  37,
  1539,
  1715,
  1553,
  1541,
  1536,
  1532,
  1872,
  1538]}

In [77]:
#take the author out 
label_au = []
for i,d in enumerate(train_data):
    author = np.array(d['authors'])
    author_100 = author[author<100]
    #length is 101, put the author number which greater than 100 in the last one in the array 
    au_range = np.zeros(101)
    if len(author_100) == 0:
        au_range[-1] = 1 
    else:
        au_range[author_100] = 1
    label_au.append(au_range)

In [82]:
author_label_train = np.array(label_au)

In [8]:
# label the title 
label_title = []
for cont in train_data:
    t = cont['title']
    leng= len(t)
    # the longest title is 127
    title_l = np.zeros(127)
    title_l[:leng] = t
    label_title.append(title_l)

In [10]:
label_title[1]

array([1731.,   47.,   11.,   57., 4624., 1525., 1535.,   47.,   11.,
       3522., 2223., 1653.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.])

In [ ]:
# label the abstract
label_abstract = []
for cont in train_data:
    t = cont['abstract']
    leng= len(t)
    # the longest title is 127
    title_l = np.zeros(127)
    title_l[:leng] = t
    label_title.append(title_l)
#data classification
#contrastive learning
#word2vec

In [4]:
#using word2vec
#pip install --upgrade gensim
from gensim.models import Word2Vec



In [5]:
# ad title and abstract together and use word2vec as a model
title_l = []
abstract_l = []
for contect in train_data:
    ti = contect['title']
    title_l.append(ti)
    abst = contect['abstract']
    abstract_l.append(abst)

w2v_model = Word2Vec(sentences=(title_l+abstract_l),vector_size = 128,min_count = 1)
w2v = w2v_model.wv

In [7]:
w2v[1]

array([ 0.06564054, -0.02705611, -0.08554906, -0.6123442 ,  0.71550083,
        0.8263495 , -0.37299272,  0.21958807,  0.18188247,  0.28978765,
        0.52275795, -0.05760271, -0.14251505, -0.17974982, -0.50025845,
        0.02567676, -0.43265936,  0.2914084 ,  0.14193542,  0.41156778,
        0.18421966,  0.26185444, -0.37423015,  0.34316364, -0.42898917,
        0.09826511, -0.3889142 ,  0.47281328, -0.0855048 ,  0.18498467,
       -0.28912368, -0.43982643,  0.12890159, -0.1473765 ,  0.56648314,
       -0.7942524 ,  0.52610296, -0.62459743,  0.32831642,  0.02035848,
        0.47404313,  0.19665815, -0.2556333 , -0.52210385,  0.15195428,
        0.15413976, -0.24150316,  0.18039654,  0.2362605 ,  0.3372081 ,
       -0.3123496 ,  0.28255677,  0.2200401 , -0.43868232, -0.3970438 ,
        0.55873585,  0.54805243,  0.5968465 ,  0.15600488, -0.3137945 ,
        0.19650355, -0.12956019,  0.17694892, -0.02325347,  0.00151268,
       -0.15879864,  0.06274869,  0.43543872,  0.42784107,  0.01

In [6]:
vect_ta = []
for cont in train_data:
    titl = cont['title']
    abstr = cont['abstract']
    all_ta = titl+abstr
    vector = np.zeros(128)
    for wor in all_ta:
        vector +=w2v[wor]
    vector = vector/len(all_ta)
    vect_ta.append(vector)
vect_ta = np.array(vect_ta)

In [11]:
#using one hot method 
venue_l = []
for cont in train_data:
    #venue id from 0 to 464, include ' ' will have 466 id
    vn_n = np.zeros(466)
    ven = cont['venue']
    if ven == '':
        vn_n[-1] = 1
    else:
        vn_n[ven] = 1
    venue_l.append(vn_n)
venue_l = np.array(venue_l)

In [15]:
#find out of distribution and remove it, 
#it means that find out the id more than 100 but corparate with less than id's is 100
co_auth = {}
for cont in train_data:
    auth = np.array(cont['authors'])
    auth_100 = auth[auth<100]
    auth_101 = auth[auth>=100]
    if len(auth_100)!=0:
        for j in auth_101:
            co_auth[j] = 1


In [18]:
coau_l = list(co_auth.keys())

In [22]:
#using one hot vector method for co-author
co_author = []
idx_coau ={}
for d,idx in enumerate(coau_l):
    idx_coau[idx] = d
    

for conte in train_data:
    auth = np.array(cont['authors'])
    auth_100 = auth[auth<100]
    auth_101 = auth[auth>=100]
    co_n = np.zeros(6576)# len of co-author
    if len(auth_100) == 0:
        co_n[-1] = 1
    else:
        for i in auth_101:
            co_n[idx_coau[i]] =1
    co_author.append(co_n)
co_author = np.array(co_author)

In [23]:
thr_feat = np.concatenate((vect_ta,venue_l),axis =1)

In [24]:
mix_featr = np.concatenate((thr_feat ,co_author),axis =1)

In [61]:
np.save('train_d.npy',mix_featr)

(800, 7170)

In [62]:
mix_featr.shape

(25793, 7170)

In [49]:
test_data =json.load(open('test.json'))

In [50]:
vect_test = []
for cont in test_data:
    titl = cont['title']
    abstr = cont['abstract']
    all_ta = titl+abstr
    vector = np.zeros(128)
    for wor in all_ta:
        vector +=w2v[wor]
    vector = vector/len(all_ta)
    vect_test.append(vector)
vect_test = np.array(vect_test)

In [64]:
venue_test.shape

(800, 466)

In [51]:
#using one hot method 
venue_test = []
for cont in test_data:
    #venue id from 0 to 464, include ' ' will have 466 id
    vn_n = np.zeros(466)
    ven = cont['venue']
    if ven == '':
        vn_n[-1] = 1
    else:
        vn_n[ven] = 1
    venue_test.append(vn_n)
venue_test = np.array(venue_test)

In [31]:
test_data[0].keys()

dict_keys(['identifier', 'coauthors', 'year', 'abstract', 'venue', 'title'])

In [66]:
co_author_test = []
for conte in test_data:
    auth = np.array(cont['coauthors'])
    auth_100 = auth[auth<100]
    auth_101 = auth[auth>=100]
    co_n = np.zeros(6576)# len of co-author
    if len(auth_100) == 0:
        co_n[-1] = 1
    else:
        for i in auth_101:
            co_n[idx_coau[i]] = 1
    co_author_test.append(co_n)
co_author_test = np.array(co_author_test)

In [67]:
co_author_test.shape

(800, 6576)

In [68]:
thr_feat_test = np.concatenate((vect_test,venue_test),axis =1)

In [69]:
mix_featr_test = np.concatenate((thr_feat_test ,co_author_test),axis =1)

In [71]:
mix_featr_test.shape

(800, 7170)

In [72]:

np.save('test_d.npy',mix_featr_test)
#save the feature data 

In [74]:
# classification model
x =np.load('train_d.npy')
y = np.load('test_d.npy')


In [83]:
x_train,x_test,y_train,y_test = train_test_split(x,author_label_train,test_size = 0.3,random_state =1)

In [86]:
y_train.shape

(18055, 101)

In [99]:
model_all = Sequential()
model_all.add(Dense(512, input_dim = x_train.shape[1],activation= 'relu'))
model_all.add(Dropout(0.2))
model_all.add(Dense(256,activation ='relu'))
model_all.add(Dropout(0.2))
model_all.add(Dense(y_train.shape[1],activation ='sigmoid'))
model_all.compile(loss = bp_mll_loss,optimizer='adam')
model_all.fit(x_train,y_train,epochs = 50,batch_size = 128)

Epoch 1/50
142/142 [==============================] - 6s 30ms/step - loss: 0.5822
Epoch 2/50
142/142 [==============================] - 4s 30ms/step - loss: 0.5560
Epoch 3/50
142/142 [==============================] - 5s 36ms/step - loss: 0.5560
Epoch 4/50
142/142 [==============================] - 5s 34ms/step - loss: 0.5560
Epoch 5/50
142/142 [==============================] - 5s 35ms/step - loss: 0.5560
Epoch 6/50
142/142 [==============================] - 5s 33ms/step - loss: 0.5560
Epoch 7/50
142/142 [==============================] - 7s 51ms/step - loss: 0.5559
Epoch 8/50
142/142 [==============================] - 8s 59ms/step - loss: 0.5559
Epoch 9/50
142/142 [==============================] - 6s 41ms/step - loss: 0.5559
Epoch 10/50
142/142 [==============================] - 5s 34ms/step - loss: 0.5559
Epoch 11/50
142/142 [==============================] - 5s 34ms/step - loss: 0.5559
Epoch 12/50
142/142 [==============================] - 5s 38ms/step - loss: 0.5559
Epoch 13/50
1

In [100]:
predi_re = model_all.predict(x_train)

565/565 [==============================] - 2s 4ms/step


In [101]:
predi_re = np.where(predi_re >=0.5,1,0)

In [102]:
f1_score(y_true = y_train, y_pred = predi_re,average ='samples')

0.5641588863718914

In [104]:
predi_test = model_all.predict(x_test)
predi_test = np.where(predi_test >=0.5,1,0)
f1_score(y_true = y_test, y_pred = predi_test,average ='samples')

242/242 [==============================] - 1s 3ms/step


0.5299794390238037

In [108]:
predic_final = model_all.predict(y)

25/25 [==============================] - 0s 4ms/step


In [115]:
predic_final_idx = [np.where(i >=0.5) for i in predic_final]

In [116]:
csv_t = pd.DataFrame(predic_final_idx, columns =['Predict'])

In [117]:
csv_t

,Predict
0,[100]
1,[100]
2,"[56, 81, 100]"
3,"[6, 23, 31, 50, 78, 100]"
4,"[26, 100]"
...,...
795,[100]
796,"[16, 19, 21, 53, 63, 100]"
797,"[14, 74, 92, 100]"
798,"[11, 41, 43, 53, 81, 100]"


array([100])